<a href="https://colab.research.google.com/github/aruaru0/pytorch-tests/blob/main/rakutenAI_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate
#==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install gradio

In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer


MODEL_NAME = "Rakuten/RakutenAI-7B-chat"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                             device_map="auto",
                                             torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

model.eval()

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1341: UserWarning: Current model requires 536875008 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(48000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [21]:
requests = [
    "「馬が合う」はどう言う意味ですか",
    "How to make an authentic Spanish Omelette?",
]

system_message = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {user_input} ASSISTANT:"

for req in requests:
    input_req = system_message.format(user_input=req)
    input_ids = tokenizer.encode(input_req, return_tensors="pt").to(device=model.device)
    tokens = model.generate(
        input_ids,
        max_new_tokens=1024,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer = streamer
    )
    out = tokenizer.decode(tokens[0][len(input_ids[0]):], skip_special_tokens=True)
    print("USER:\n" + req)
    print("ASSISTANT:\n" + out)
    print()
    print()

「馬が合う」は、相性が良い、息が合う、協力しやすい、といった意味を持つ成句です。馬に乗る際には、馬との息を揃えなければならないため、「馬が合う」という成句は、相手と息が合う、協力しやすいといった意味で使われます。「二人(ふたり)で馬を借りたなら、馬が合うようにすまえ」という諺があるように、うまくいくためには相性と息が合うことが重要だとされています。ビジネスの世界でも、上司と部下の馬が合わなければ、仕事でうまく進まず、良好な関係を維持することが難しくなってしまいます。
input_req
USER:
「馬が合う」はどう言う意味ですか
ASSISTANT:
「馬が合う」は、相性が良い、息が合う、協力しやすい、といった意味を持つ成句です。馬に乗る際には、馬との息を揃えなければならないため、「馬が合う」という成句は、相手と息が合う、協力しやすいといった意味で使われます。「二人(ふたり)で馬を借りたなら、馬が合うようにすまえ」という諺があるように、うまくいくためには相性と息が合うことが重要だとされています。ビジネスの世界でも、上司と部下の馬が合わなければ、仕事でうまく進まず、良好な関係を維持することが難しくなってしまいます。




In [22]:
input_req

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: 「馬が合う」はどう言う意味ですか ASSISTANT:"

# Gradioでチャットボットを作ってみる

ストリームを使いたかったので以下を参考にした

https://zenn.dev/platina/articles/e6aa6e96373c09

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer


MODEL_NAME = "Rakuten/RakutenAI-7B-chat"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                             device_map="auto",
                                             torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

model.eval()

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1341: UserWarning: Current model requires 536875008 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(48000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

## プロンプトを作成

In [3]:
SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。\n"

def compose_prompt(message:str, history:list[list[str]]) -> str:
    prompt = f"{SYSTEM_PROMPT}"
    if len(history) == 0:
        prompt += f"USER:{message}\nASSISTANT:"
        return prompt
    else:
        for pair in history:
            [user, assistant] = pair
            prompt += f"USER: {user} \nASSISTANT:{assistant} <|endoftext|>\n"

        prompt += f"USER:{message}\nASSISTANT:"
        return prompt


# プロンプトを確認
prompt = compose_prompt("日本で一番高い山は？", [["一番長い川は？", "信濃川です"]])
print(prompt)

あなたは誠実で優秀な日本人のアシスタントです。
USER: 一番長い川は？ 
ASSISTANT:信濃川です <|endoftext|>
USER:日本で一番高い山は？
ASSISTANT:


## ストリーマーを作成

In [4]:
from threading import Thread

async def gen_stream(prompt: str) -> TextIteratorStreamer:
    input_ids = tokenizer(prompt, return_tensors="pt").to(device=model.device)

    # print(prompt, input_ids)
    config = dict(
        **input_ids,
        max_new_tokens=512,
        streamer=streamer,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
    )

    thread = Thread(target=model.generate, kwargs=config)
    thread.start()

    return streamer

## GradioでGUIを作成

In [5]:
import gradio as gr

async def chat(message: str, history: list[list[str]]):
    prompt = compose_prompt(message, history)

    st = await gen_stream(prompt)

    total_response = ""

    for output in st:
        if not output:
            continue

        total_response += output

        yield total_response

demo = gr.ChatInterface(chat).queue()
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a9eb5428cfd07fa180.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
